# Процессы и потоки (Лабораторная работа №2)
## Выполнил: студент группы БВТ2001 Хайдаров Адель Фанисович
##### Цель работы: получение практических навыков использования Win32 API для исследования процессов

### Вспомогательные функции

In [ ]:
open System.Runtime.InteropServices
open System.Text

type dw = uint
type lps = StringBuilder
type hdl = nativeint

let mutable maxLength = 260
let disp (handle: hdl) = display $"&H%x{handle}"

### Функции Win32 API
Ниже представлены использованные функции Win32 API для получения той или иной информации о системе и для задания некоторых параметров

In [ ]:
type Access = | Terminate = 1 | Create = 2
type Snaps = | HeapList = 1 | Process = 2 | Thread = 4 | Module = 8 | All = 15
[<DllImport("kernel32.dll")>] extern dw GetCurrentProcessId()
[<DllImport("kernel32.dll")>] extern dw GetModuleFileName(hdl md, lps file, int size)
[<DllImport("kernel32.dll")>] extern hdl GetCurrentProcess()
[<DllImport("kernel32.dll")>] extern hdl OpenProcess(Access acc, bool inheritHandle, dw OpenProcess)
[<DllImport("kernel32.dll")>] extern hdl CreateToolhelp32Snapshot(Snaps flags, dw pid)
[<DllImport("kernel32.dll")>] extern hdl CreateMutex(string sec, bool isInitial, string name)
[<DllImport("kernel32.dll")>] extern bool GetModuleHandleEx(dw flags, string name, hdl& ph)
[<DllImport("kernel32.dll")>] extern bool CloseHandle(hdl object)
[<DllImport("kernel32.dll")>]
extern void DuplicateHandle(hdl sp, hdl src, hdl tp, hdl& target, dw access, bool inheritHandle, dw options)

type ProcessEntry = struct
    val mutable size: dw
    val usage: dw; val pid: dw; val heapId: nativeint; val moduleId: dw
    val threads: dw; val parentId: dw; val priority: dw; val flags: dw
    [<MarshalAs(UnmanagedType.ByValTStr, SizeConst = 260)>]
    val executable: string
end
[<DllImport("kernel32.dll")>] extern bool Process32Next(hdl snap, ProcessEntry& entry)

type ThreadEntry = struct
    val mutable size: dw
    val usage: dw; val tid: dw; val ownerId: dw; val basePriority: int
    val obs: int; val flags: dw
end
[<DllImport("kernel32.dll")>] extern bool Thread32Next(hdl snap, ThreadEntry& entry)

type ModuleEntry = struct
    val mutable size: dw
    val mid: dw; val pid: dw; val usage: dw; val procUsage: dw
    val baseAddr: nativeint; val baseSize: dw; val hm: hdl
    [<MarshalAs(UnmanagedType.ByValTStr, SizeConst = 256)>]
    val xModule: string
    [<MarshalAs(UnmanagedType.ByValTStr, SizeConst = 260)>]
    val executable: string
end
[<DllImport("kernel32.dll")>] extern bool Module32Next(hdl snap, ModuleEntry& entry)

let defaultPEntry = ProcessEntry(size = uint (sizeof<ProcessEntry> + maxLength * 2))
let defaultTEntry = ThreadEntry(size = uint sizeof<ThreadEntry>)
let defaultMEntry = ModuleEntry(size = uint sizeof<ModuleEntry> + (256u + 260u) * 2u)

let getHandle name =
    let mutable target = hdl 0
    if GetModuleHandleEx(0u, name, &target) |> not then failwith("No handle discovered")
    target

let getFilename handle =
    let mutable sb = lps maxLength
    if GetModuleFileName(handle, sb, maxLength) = 0u then failwith("Can't locate the executable")
    sb.ToString()

let duplicateHdl cph =
    let mutable target = hdl 0
    let mutex = CreateMutex(null, false, null)
    DuplicateHandle(cph, mutex, cph, &target, 0u, false, 0u)
    target

### Выполнение

#### Задание 1

In [ ]:
let h1, h2 = getHandle "user32", getHandle "dotnet.exe"

display $"Для Name='user32':\nHandle: &H%x{h1}, FName: {h1 |> getFilename}"
display $"Для Name='dotnet.exe':\nHandle: &H%x{h2}, FName: {h2 |> getFilename}"
display $"FName текущего процесса: {getFilename 0}"

Для Name='user32':
Handle: &H7ffefa710000, FName: C:\Windows\System32\USER32.dll

Для Name='dotnet.exe':
Handle: &H7ff6ffc80000, FName: C:\Program Files\dotnet\dotnet.exe

FName текущего процесса: C:\Program Files\dotnet\dotnet.exe

#### Задание 2

In [ ]:
let id, ph = GetCurrentProcessId(), GetCurrentProcess()
let real = duplicateHdl ph
let gp = OpenProcess(Access.Terminate, true, id)

display $"Идентификатор текущего процесса: {id}"
display $"Псевдодескриптор текущего процесса: {ph}"
display $"Дескриптор текущего процесса: {real}"
display $"Открытый дескриптор текущего процесса: {gp}"

// Закрытие дескрипторов
for j in [real; gp] do if CloseHandle j then display $"{j} закрыт" |> ignore

Идентификатор текущего процесса: 2944

Псевдодескриптор текущего процесса: -1

Дескриптор текущего процесса: 3664

Открытый дескриптор текущего процесса: 3660

3664 закрыт

3660 закрыт

#### Задание 3

Процессы:

In [ ]:
let mutable snap = CreateToolhelp32Snapshot(Snaps.Process, 0u)
let mutable entry, hasNext = defaultPEntry, true
seq {
    while hasNext do
        hasNext <- Process32Next(snap, &entry)
        entry
}

index,usage,pid,heapId,moduleId,threads,parentId,priority,flags,executable,size
0,0,0,0,0,8,0,0,0,[System Process],568
1,0,4,0,0,201,0,8,0,System,568
2,0,72,0,0,0,4,8,0,Secure System,568
3,0,128,0,0,4,4,8,0,Registry,568
4,0,424,0,0,2,4,11,0,smss.exe,568
5,0,680,0,0,11,540,13,0,csrss.exe,568
6,0,760,0,0,1,540,13,0,wininit.exe,568
7,0,768,0,0,12,752,13,0,csrss.exe,568
8,0,856,0,0,3,752,13,0,winlogon.exe,568
9,0,876,0,0,6,760,9,0,services.exe,568


Потоки:

In [ ]:
let mutable snap = CreateToolhelp32Snapshot(Snaps.Thread, 0u)
let mutable entry, hasNext = defaultTEntry, true
seq {
    while hasNext do
        hasNext <- Thread32Next(snap, &entry)
        entry
}

index,usage,tid,ownerId,basePriority,obs,flags,size
0,0,0,0,0,0,0,28
1,0,0,0,0,0,0,28
2,0,0,0,0,0,0,28
3,0,0,0,0,0,0,28
4,0,44,0,0,0,0,28
5,0,52,0,0,0,0,28
6,0,60,0,0,0,0,28
7,0,68,0,0,0,0,28
8,0,12,4,13,0,0,28
9,0,16,4,13,0,0,28


Модули:

In [ ]:
let mutable snap = CreateToolhelp32Snapshot(Snaps.Module, 0u)
let mutable entry, hasNext = defaultMEntry, true
seq {
    while hasNext do
        hasNext <- Module32Next(snap, &entry)
        entry
}

index,mid,pid,usage,procUsage,baseAddr,baseSize,hm,xModule,executable,size
0,1,0,65535,65535,140698829979648,151552,140698829979648,dotnet.exe,C:\Program Files\dotnet\dotnet.exe,1088
1,1,0,65535,65535,140733118545920,2134016,140733118545920,ntdll.dll,C:\Windows\SYSTEM32\ntdll.dll,1088
2,1,0,65535,65535,140733102424064,774144,140733102424064,KERNEL32.DLL,C:\Windows\System32\KERNEL32.DLL,1088
3,1,0,65535,65535,140733076144128,3653632,140733076144128,KERNELBASE.dll,C:\Windows\System32\KERNELBASE.dll,1088
4,1,0,1,1,140733079814144,1118208,140733079814144,ucrtbase.dll,C:\Windows\System32\ucrtbase.dll,1088
5,1,0,65535,65535,140730624770048,389120,140730624770048,hostfxr.dll,C:\Program Files\dotnet\host\fxr\6.0.5\hostfxr.dll,1088
6,1,0,65535,65535,140733103865856,712704,140733103865856,ADVAPI32.dll,C:\Windows\System32\ADVAPI32.dll,1088
7,1,0,65535,65535,140733098098688,667648,140733098098688,msvcrt.dll,C:\Windows\System32\msvcrt.dll,1088
8,1,0,65535,65535,140733103210496,647168,140733103210496,sechost.dll,C:\Windows\System32\sechost.dll,1088
9,1,0,65535,65535,140733098819584,1179648,140733098819584,RPCRT4.dll,C:\Windows\System32\RPCRT4.dll,1088


Драйверы устройств:

In [ ]:
[<DllImport("psapi")>]
extern bool EnumDeviceDrivers(
    [<MarshalAs(UnmanagedType.LPArray, ArraySubType = UnmanagedType.U4)>] dw[] fr,
    dw size, dw& needed
)

let mutable nb: dw = 0u
EnumDeviceDrivers(null, 0u, &nb) |> ignore

let newSize = nb / 4u
let mutable drivers: dw [] = int <| newSize |> Array.zeroCreate

EnumDeviceDrivers(drivers, newSize, &nb)

for t in drivers do if t <> 0u then printfn $"{t}"

1574961152
4294965248
1512177664
4294965248
1512243200
4294965248
1512505344
4294965248
1512308736
4294965248
1512964096
4294965248
1513095168
4294965248
1514340352
4294965248
1515061248
4294965248
1514864640
4294965248
1513160704
4294965248
1515520000
4294965248
1515585536
4294965248
1515716608
4294965248
1608515584
4294965248
1609498624
4294965248
1610285056
4294965248
1515782144
4294965248
1515913216
4294965248
1515978752
4294965248
1611202560
4294965248
1611333632
4294965248
1611726848
4294965248
1611857920
4294965248
1611923456
4294965248
1613103104
4294965248
1613955072
4294965248
1614020608
4294965248
1614086144
4294965248
1614675968
4294965248
1615200256
4294965248
1615724544
4294965248
1615855616
4294965248
1615986688
4294965248
1616052224
4294965248
1616183296
4294965248
1616314368
4294965248
1616510976
4294965248
1616642048
4294965248
1616904192
4294965248
1617756160
4294965248
1617887232
4294965248
1618345984
4294965248
1618477056
4294965248
1619656704
4294965248
1620639744

# Вывод

В результате лабораторной работы мной была проведена работа с потоками и процессами, используя средства Win32 API.